In [1]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import datetime as dt

data = pd.read_csv('D:\DataSci\Resources\historical_automobile_sales.csv')
app = dash.Dash(__name__)
app.layout = html.Div(children=[
    (html.H1('Automobile Sales Statistics Dashboard', style={'textAlign':'center', 'color':'#503D36', 'font-size':24})),
    dcc.Dropdown(id='dropdown-statistics',options=[{'label':'Yearly Statistics','value':'Yearly Statistics' },{'label':'Recession Period Statistics','value':'Recession Period Statistics' }], placeholder='Select a report type', value='Select Statistics'),
    dcc.Dropdown(id='select-year', options=[{'label':i, 'value':i }for i in range(1980,2024)], placeholder='Select-year', value='Select-year'),
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display':'flex'})
    ])
])


@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics',component_property='value'))
def update_input_container(report_type):
    if report_type =='Yearly Statistics': 
        return False
    else: 
        return True
    

@app.callback(Output(component_id='output-container', component_property='children'),
              [Input(component_id='dropdown-statistics',component_property='value'),
               Input(component_id='select-year', component_property='value')])

def update_output_container(report_type, selected_year):
    
    if report_type == 'Recession Period Statistics':
        recession_data = data.loc[data['Recession']==1]
        
        #plot-1
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart_1 = dcc.Graph(
            figure=px.line(yearly_rec,x='Year',y='Automobile_Sales', title='Automobile sales fluctuation over Recession Period (year wise)')
        )

        #plot-2
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart_2= dcc.Graph(
            figure = px.bar(average_sales,x='Vehicle_Type',y='Automobile_Sales', title='Average number of vehicles sold by vehicle type')
        )
    
        #plot-3
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart_3 = dcc.Graph(
            figure=px.pie(exp_rec, names='Vehicle_Type', values='Advertising_Expenditure', title='Total expenditure share by vehicle type')
        )

        #plot-4
        unemp_data  = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart_4 = dcc.Graph(
            figure = px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales', labels={'unemployment_rate':'Unemployment Rate', 'Automobile_Sales':'Automobile Sales'}, title='Effect of Unemployment Rate on Vehicle Type and Sales')
        )

        return[
            html.Div(className='chart-item', children=[html.Div(children=R_chart_1),html.Div(children=R_chart_2)],style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart_3),html.Div(children=R_chart_4)],style={'display': 'flex'})
        ]

    elif(report_type=='Yearly Statistics' and selected_year):
        yearly_data = data[data['Year'] == selected_year]
        #plot-1
        yearly_rec = yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart_1 = dcc.Graph(
            figure=px.line(yearly_rec,x='Year',y='Automobile_Sales', title='Automobile sales fluctuation over Recession Period (year wise)')
        )
        
        #plot-2
        average_sales = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart_2= dcc.Graph(
            figure = px.bar(average_sales,x='Vehicle_Type',y='Automobile_Sales', title='Average number of vehicles sold by vehicle type')
        )

        #plot-3
        exp_rec = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart_3 = dcc.Graph(
            figure=px.pie(exp_rec, names='Vehicle_Type', values='Advertising_Expenditure', title='Total expenditure share by vehicle type')
        )

        #plot-4
        unemp_data  = yearly_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        Y_chart_4 = dcc.Graph(
            figure = px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales', labels={'unemployment_rate':'Unemployment Rate', 'Automobile_Sales':'Automobile Sales'}, title='Effect of Unemployment Rate on Vehicle Type and Sales')
        )

        
        return[
            html.Div(className='chart-item', children=[html.Div(children=Y_chart_1),html.Div(children=Y_chart_2)],style={'display':'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart_3),html.Div(children=Y_chart_4) ],style={'display': 'flex'})
        ]





        

if __name__ == '__main__':
    app.run_server()